In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier #knn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [39]:
from sklearn.preprocessing import LabelEncoder

In [21]:
data= pd.read_csv("C:\\Users\\MASSIVE\\Price-Predictor-ML-\\final.csv")

In [22]:
print(data.columns)
data.isnull().sum()

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'],
      dtype='object')


id                          0
title                       0
price                       0
specs_annee                 0
specs_car_engine         9798
specs_couleur_auto       2494
specs_papiers            4024
specs_kilometrage        4661
specs_marque-voiture        0
specs_modele                1
specs_finition          18647
specs_energie           92547
specs_boite             92547
option0                 17057
option1                 21209
option2                 23907
option3                 26781
option4                 29897
option5                 33568
option6                 37328
option7                 41555
option8                 46047
option9                 50978
option10                56118
option11                61725
option12                68319
option13                77359
dtype: int64

In [23]:
data.describe()

,id,price,specs_annee,specs_energie,specs_boite
count,9.254700e+04,9.254700e+04,92547.000000,0.0,0.0
mean,3.812892e+07,1.203689e+17,2012.852723,NaN,NaN
std,3.494813e+06,3.307373e+19,8.425591,NaN,NaN
min,5.065853e+06,0.000000e+00,1922.000000,NaN,NaN
25%,3.731984e+07,3.000000e+04,2010.000000,NaN,NaN
50%,3.892210e+07,1.330000e+06,2014.000000,NaN,NaN
75%,4.049664e+07,3.000000e+06,2019.000000,NaN,NaN
max,4.114435e+07,1.000000e+22,2024.000000,NaN,NaN


In [24]:
df_sans_redondance = data.drop_duplicates(subset=['title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'])

In [25]:
df_sans_redondance.describe()

,id,price,specs_annee,specs_energie,specs_boite
count,8.165000e+04,8.165000e+04,81650.000000,0.0,0.0
mean,3.801312e+07,1.364192e+17,2012.781115,NaN,NaN
std,3.610762e+06,3.521164e+19,8.432651,NaN,NaN
min,5.065853e+06,0.000000e+00,1922.000000,NaN,NaN
25%,3.725727e+07,3.000000e+04,2009.000000,NaN,NaN
50%,3.878532e+07,1.310000e+06,2014.000000,NaN,NaN
75%,4.046483e+07,3.000000e+06,2019.000000,NaN,NaN
max,4.114435e+07,1.000000e+22,2024.000000,NaN,NaN


In [26]:
copy=df_sans_redondance.copy()

In [27]:
# Create a new column named 'new_options' and initialize it with 0
copy['options'] = 0

# Iterate over each row
for index, row in copy.iterrows():
    options_count = 0
    # Count options from option0 to option13
    for i in range(14):
        option_col = f'option{i}'
        if not pd.isnull(row[option_col]):
            options_count += 1
    # Store the count of options in the 'new_options' column
    copy.at[index, 'options'] = options_count

In [28]:
copy.columns

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13', 'options'],
      dtype='object')

In [29]:
copy['options'].value_counts

<bound method IndexOpsMixin.value_counts of 0         0
1         1
2         7
3        11
4         0
         ..
92542    14
92543    14
92544     0
92545     7
92546    10
Name: options, Length: 81650, dtype: int64>

In [30]:
copy.drop(['option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'], axis=1,inplace=True)

In [31]:
copy.columns

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options'],
      dtype='object')

In [32]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite']].dtypes)

id                        int64
title                    object
price                   float64
specs_annee               int64
specs_car_engine         object
specs_couleur_auto       object
specs_papiers            object
specs_kilometrage        object
specs_marque-voiture     object
specs_modele             object
specs_finition           object
specs_energie           float64
specs_boite             float64
dtype: object


In [33]:
copy['specs_kilometrage'] = pd.to_numeric(copy['specs_kilometrage'].str.replace('km', ''), errors='coerce')

In [34]:
copy['specs_annee'] = pd.to_numeric(copy['specs_annee'],errors='coerce')
copy['price'] = pd.to_numeric(copy['price'],errors='coerce')

In [35]:
copy['specs_boite'].value_counts()

Series([], Name: count, dtype: int64)

In [36]:
# Check for null values in the 'specs_energie' column
null_count = copy['specs_boite'].isnull().sum()
print("Number of null values:", null_count)

# Display unique values in the 'specs_energie' column
unique_values = copy['specs_boite'].unique()
print("Unique values:", unique_values)


Number of null values: 81650
Unique values: [nan]


In [37]:
copy['specs_papiers'] = copy['specs_papiers'].replace({
    'Carte grise / safia': 1,
    'Licence / Délai': 2,
    'Carte jaune': 3,
})

# Display the resulting DataFrame
print(copy['specs_papiers'].value_counts())

specs_papiers
1.0    73675
2.0     2875
3.0     1527
Name: count, dtype: int64


In [38]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options']].dtypes)

id                        int64
title                    object
price                   float64
specs_annee               int64
specs_car_engine         object
specs_couleur_auto       object
specs_papiers           float64
specs_kilometrage       float64
specs_marque-voiture     object
specs_modele             object
specs_finition           object
specs_energie           float64
specs_boite             float64
options                   int64
dtype: object


In [42]:
label_encoder = LabelEncoder()
copy['specs_couleur_auto'] = label_encoder.fit_transform(copy['specs_couleur_auto'])

In [47]:
copy['specs_marque-voiture'].value_counts()

specs_marque-voiture
Renault                  14962
Peugeot                  10856
Volkswagen                9256
Hyundai                   4874
Seat                      4394
                         ...  
Renault laguna               1
Dacia stpway sandero         1
Fiat une                     1
Renu kongo                   1
Ds7                          1
Name: count, Length: 634, dtype: int64

In [76]:
# Mapping dictionary to standardize brand names
brand_mapping = {
    'Renault laguna': 'Renault','رنوا': 'Renault','R4': 'Renault','Renu kongo':'Renault',
    'Dacia stpway sandero': 'Dacia','Dacia stpway sandero':'Dacia','Dacia stpway sandero':'Dacia',
    'Fiat une': 'Fiat','FIAT TIPO': 'Fiat',
    'Renu kongo': 'Renault','كليو': 'Renault',
    'Ds7': 'DS Automobiles',
    'Picanto' : 'Kia','Kia  sportage': 'Kia',
    'Ikco': 'Other',  # Correcting 'Ikco' to 'Other' (assuming it's not a valid brand)
    'chevroli': 'Chevrolet',  # Correcting 'chevroli' to 'Chevrolet'
    'Cheri': 'Chery','chiri': 'Chery',  # Correcting 'Cheri' to 'Chery'
    'DS Automobiles': 'DS' ,
    'ايبيزا 2015 2 بورت': 'Seat',
    'bmw e36':'BMW',
    'Picasso':'Citroen'
}

# Replace inconsistent brand names with the correct ones
copy['specs_marque-voiture'].replace(brand_mapping, inplace=True)

In [75]:
# Check the updated value counts
print(copy['specs_marque-voiture'].value_counts())

specs_marque-voiture
Renault                  14966
Peugeot                  10856
Volkswagen                9256
Hyundai                   4874
Seat                      4395
                         ...  
Dacia stpway sandero         1
Renu kongo                   1
NEW king                     1
Hors bord rocca              1
Hibao                        1
Name: count, Length: 619, dtype: int64


In [77]:
unique_brands = copy['specs_marque-voiture'].unique()

# Print the list of unique brands
print(unique_brands)

['Dacia' 'Seat' 'Kia' 'Nissan' 'Volkswagen' 'Peugeot' 'Fiat' 'Renault'
 'Chery' 'Great Wall' 'Citroen' 'Hyundai' 'Suzuki' 'Alfa Romeo'
 'Chevrolet' 'Mercedes' 'BMW' 'DS' 'Toyota' 'Huanghai' 'Vms' 'Skoda'
 'Daewoo' 'Balingo' 'Haima' 'Zotye' 'Audi' 'Mini Cooper' 'Gonow'
 'Land Rover' 'DFSK' 'Ibiza' 'SsangYong' 'QQ' 'JAC' 'Ford' 'Lada' 'Chana'
 'Opel' 'Baic' 'Honda' 'MG' 'CERTA' 'Faw' 'Chrysler' 'DFM'
 'Fiat Professional' 'Daihatsu' 'Hafei motors' 'Sonacom' 'Isuzu' 'Lagona'
 'Pougou' 'Desactivation adblue sur kia sportage et hyundai tucson'
 'رونو 18براك' 'Geely' 'Mazda' 'Kango' 'Pertner'
 'Yamaha svho super charge' 'Lifan' 'Maruti' 'Ford fiesta' 'Tata'
 'Gonow dfsk dfm sokon chana hafei faw' 'Rover' 'Mitsubishi' 'Changan'
 'clio' 'Porsche' 'رونو   18' 'Aston Martin' 'Ssang yong' 'Tesla' 'Atos'
 'Brilliance' 'رونو' 'Maserati' 'Xsara' 'Smbole' 'Jaguar' 'بيجو' 'Sym'
 'terios' 'Dfsk dfm' 'فورد' 'شفرولي افيو' 'Golf' 'Partner' 'toureg' 'Cuxi'
 'Renault Kangoo' 'Victoria' 'Cadillac' 'Doblo' 'Cl